In [693]:
import pandas as pd
import sqlite3
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

In [694]:
connEmpleados = sqlite3.connect('data/empleados.db')
connVacaciones = sqlite3.connect('data/vacaciones.db')
connDepartamentos = sqlite3.connect('data/departamentos.db')

In [695]:
dfEmpleados = pd.read_sql_query('SELECT * FROM empleados', connEmpleados)
dfDepartamentos = pd.read_sql_query('SELECT * FROM departamentos', connDepartamentos)
dfVacaciones = pd.read_sql_query('SELECT * FROM vacaciones', connVacaciones)

In [696]:
# Cerrar las conexiones a las bases de datos
connEmpleados.close()
connDepartamentos.close()
connVacaciones.close()

## --- Modelo de Predicción Aprobación/Rechazo ---

In [697]:
# Fusionar todos los datos
df_merged = pd.merge(dfVacaciones, dfEmpleados, left_on='id_empleado', right_on='id_empleado', how='left')

df_modelo = pd.merge(df_merged, dfDepartamentos, left_on='id_departamento', right_on='id_departamento', how='left', suffixes=('_empleado', '_departamento'))

In [698]:
# Codificar variables categóricas
df_modelo['es_rol_critico_encoded'] = df_modelo['es_rol_critico'].map({'Sí': 1, 'No': 0}).fillna(0)
df_modelo['tiene_ausencias_frecuentes_encoded'] = df_modelo['tiene_ausencias_frecuentes'].map({'Sí': 1, 'No': 0}).fillna(0)
df_modelo['estado_solicitud_encoded'] = df_modelo['estado_solicitud'].map({'aprobada': 1, 'rechazada': 0})

In [699]:
df_modelo

,id_empleado,vacaciones_inicio,vacaciones_fin,fecha_solicitud,dias_solicitados,estado_solicitud,motivo_rechazo,prioridad,nombre_completo,edad_empleado,...,tiene_ausencias_frecuentes,minusvalia,activo,id_departamento,nombre_departamento,porcentaje_maximo_inactivos,numero_total_empleados,es_rol_critico_encoded,tiene_ausencias_frecuentes_encoded,estado_solicitud_encoded
0,47,2025-03-27,2025-04-02,2025-03-12,6,aprobada,None,20.05,Remigio Sandoval Barco,31,...,No,Sí,Sí,2,Ventas,15,18,0,0,1
1,60,2025-02-07,2025-02-23,2024-11-30,16,aprobada,None,30.65,Pilar Batlle Méndez,60,...,Sí,Sí,No,1,Recursos Humanos,10,24,0,1,1
2,45,2025-04-21,2025-04-22,2025-04-13,1,rechazada,Falta de personal en esas fechas,48.05,Beatriz Chamorro Cervera,50,...,No,Sí,No,2,Ventas,15,18,0,0,0
3,126,2025-04-02,2025-04-20,2025-03-26,18,aprobada,None,19.85,Bonifacio Montserrat Rodríguez,48,...,No,No,Sí,3,Marketing,10,18,1,0,1
4,49,2025-03-26,2025-04-12,2025-03-18,17,aprobada,None,25.85,Aroa Bayo Yuste,55,...,No,No,Sí,7,Logística,23,24,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,55,2025-03-07,2025-03-20,2025-02-23,13,aprobada,None,26.70,Bartolomé Blazquez Ballesteros,37,...,No,No,Sí,4,Desarrollo de Producto,15,19,0,0,1
99996,134,2024-09-06,2024-09-17,2024-07-29,11,aprobada,None,45.40,Teodoro Bárcena Matas,46,...,Sí,No,No,7,Logística,23,24,0,1,1
99997,97,2024-12-14,2024-12-27,2024-08-17,13,aprobada,None,20.80,Balduino Vera Alcalde,38,...,No,Sí,No,5,Atención al Cliente,20,23,1,0,1
99998,99,2025-03-25,2025-04-06,2025-01-05,12,rechazada,Empleado con historial de ausencias frecuentes,10.45,Ignacio Recio Quintanilla,43,...,No,No,Sí,4,Desarrollo de Producto,15,19,0,0,0


In [700]:
# Seleccionar características para el modelo de aprobación/rechazo
columnas_decision = [
    'dias_solicitados', 'prioridad', 'antiguedad_anos', 'personas_a_cargo',
    'es_rol_critico_encoded', 'tiene_ausencias_frecuentes_encoded'
]

In [701]:
# División de datos en entrenamiento y prueba
X_decision = df_modelo[columnas_decision].fillna(0)
y_decision = df_modelo['estado_solicitud_encoded']

In [702]:
# Dividir datos para el modelo de aprobación/rechazo
X_train_decision, X_test_decision, y_train_decision, y_test_decision = train_test_split(
    X_decision, y_decision, test_size=0.3, random_state=42, stratify=y_decision
)

In [703]:
# Escalar características
scaler_decision = MinMaxScaler()
X_train_scaled_decision = scaler_decision.fit_transform(X_train_decision)
X_test_scaled_decision = scaler_decision.transform(X_test_decision)

In [704]:
# Entrenar modelo de aprobación/rechazo (Gradient Boosting con GridSearchCV)
param_grid_decision = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5]
}
grid_search_decision = GridSearchCV(GradientBoostingClassifier(random_state=42), param_grid_decision, cv=3, scoring='accuracy')
grid_search_decision.fit(X_train_scaled_decision, y_train_decision)
modelo_decision = grid_search_decision.best_estimator_

In [705]:
# Evaluar modelo de aprobación/rechazo
y_pred_decision = modelo_decision.predict(X_test_scaled_decision)
accuracy_decision = accuracy_score(y_test_decision, y_pred_decision)
report_decision = classification_report(y_test_decision, y_pred_decision)

print("\n--- Resultados del Modelo de Aprobación/Rechazo ---")
print(f"Mejores hiperparámetros: {grid_search_decision.best_params_}")
print(f"Accuracy del Modelo de Decisión: {accuracy_decision:.4f}")
print("Classification Report del Modelo de Decisión:\n", report_decision)


--- Resultados del Modelo de Aprobación/Rechazo ---
Mejores hiperparámetros: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200}
Accuracy del Modelo de Decisión: 0.6240
Classification Report del Modelo de Decisión:
               precision    recall  f1-score   support

           0       0.61      0.78      0.69     15834
           1       0.65      0.45      0.53     14166

    accuracy                           0.62     30000
   macro avg       0.63      0.61      0.61     30000
weighted avg       0.63      0.62      0.61     30000



## --- Modelo de Predicción del Motivo Rechazo ---

In [706]:
# Seleccionamos registros con estado_solicitud == 'rechazada'
df_rechazadas = df_modelo[df_modelo['estado_solicitud'] == 'rechazada'].copy()
df_rechazadas = df_rechazadas.dropna(subset=['motivo_rechazo'])

In [707]:
# Codificar la variable 'motivo_rechazo'
label_encoder_motivo = LabelEncoder()
df_rechazadas['motivo_rechazo_encoded'] = label_encoder_motivo.fit_transform(df_rechazadas['motivo_rechazo'])

In [708]:
# Seleccionar características para el modelo de motivo de rechazo (añadiendo más)
columnas_motivo = [
    'dias_solicitados', 'prioridad', 'antiguedad_anos', 'personas_a_cargo',
    'es_rol_critico_encoded', 'tiene_ausencias_frecuentes_encoded'
]
X_motivo = df_rechazadas[columnas_motivo].fillna(0)
y_motivo = df_rechazadas['motivo_rechazo_encoded']

In [709]:
# Dividir datos para el modelo de motivo de rechazo
X_motivo_train, X_motivo_test, y_motivo_train, y_motivo_test = train_test_split(
    X_motivo, y_motivo, test_size=0.3, random_state=42, stratify=y_motivo
)

In [710]:
# Escalar características
scaler_motivo = MinMaxScaler()
X_motivo_train_scaled = scaler_motivo.fit_transform(X_motivo_train)
X_motivo_test_scaled = scaler_motivo.transform(X_motivo_test)

In [711]:
# Entrenar modelo de motivo de rechazo (Random Forest con GridSearchCV)
param_grid_motivo = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 3],
    'class_weight': ['balanced']
}
grid_search_motivo = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_motivo, cv=3, scoring='f1_weighted') # Usar f1_weighted para desequilibrio
grid_search_motivo.fit(X_motivo_train_scaled, y_motivo_train)
modelo_motivo_rechazo = grid_search_motivo.best_estimator_

In [712]:
# Evaluar modelo de motivo de rechazo
y_motivo_pred = modelo_motivo_rechazo.predict(X_motivo_test_scaled)
accuracy_motivo = accuracy_score(y_motivo_test, y_motivo_pred)
report_motivo = classification_report(y_motivo_test, y_motivo_pred, zero_division=0)

print("\n--- Resultados del Modelo de Predicción del Motivo de Rechazo ---")
print(f"Mejores hiperparámetros: {grid_search_motivo.best_params_}")
print(f"Accuracy del Modelo de Motivo de Rechazo: {accuracy_motivo:.4f}")
print("Classification Report del Modelo de Motivo de Rechazo:\n", report_motivo)


--- Resultados del Modelo de Predicción del Motivo de Rechazo ---
Mejores hiperparámetros: {'class_weight': 'balanced', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Accuracy del Modelo de Motivo de Rechazo: 0.4578
Classification Report del Modelo de Motivo de Rechazo:
               precision    recall  f1-score   support

           0       0.59      0.58      0.59      1649
           1       0.30      0.35      0.32      1199
           2       0.12      0.06      0.08       988
           3       0.16      0.13      0.15       963
           4       0.14      0.09      0.11       964
           5       0.58      0.50      0.54      4176
           6       0.54      0.80      0.65      4232
           7       0.09      0.05      0.06       698
           8       0.14      0.11      0.12       966

    accuracy                           0.46     15835
   macro avg       0.30      0.30      0.29     15835
weighted avg       0.42      0.46      

## --- Función de Predicción Unificada ---

In [719]:
def predecir_estado_y_motivo(nueva_solicitud, modelo_decision, scaler_decision, modelo_motivo, scaler_motivo, label_encoder_motivo, columnas_decision, columnas_motivo):
    nueva_solicitud_decision = {k: nueva_solicitud.get(k, 0) for k in columnas_decision}
    nueva_solicitud_df_decision = pd.DataFrame([nueva_solicitud_decision])
    nueva_solicitud_scaled_decision = scaler_decision.transform(nueva_solicitud_df_decision)
    prediccion_decision = modelo_decision.predict(nueva_solicitud_scaled_decision)[0]
    estado = 'aprobada' if prediccion_decision == 1 else 'rechazada'
    motivo_rechazo_predicho = None

    if estado == 'rechazada':
        nueva_solicitud_motivo = {k: nueva_solicitud.get(k, 0) for k in columnas_motivo}
        nueva_solicitud_df_motivo = pd.DataFrame([nueva_solicitud_motivo])
        nueva_solicitud_scaled_motivo = scaler_motivo.transform(nueva_solicitud_df_motivo)
        prediccion_motivo_encoded = modelo_motivo.predict(nueva_solicitud_scaled_motivo)[0]
        motivo_rechazo_predicho = label_encoder_motivo.inverse_transform([prediccion_motivo_encoded])[0]

    return estado, motivo_rechazo_predicho

# Ejemplo de predicción
nueva_solicitud_ejemplo = {
    'dias_solicitados': 34,
    'prioridad': 3,
    'antiguedad_anos': 5,
    'personas_a_cargo': 2,
    'es_rol_critico': 'Sí',
    'tiene_ausencias_frecuentes': 'No'
}

estado_predicho, motivo_predicho = predecir_estado_y_motivo(
    nueva_solicitud_ejemplo,
    modelo_decision,
    scaler_decision,
    modelo_motivo_rechazo,
    scaler_motivo,
    label_encoder_motivo,
    columnas_decision,
    columnas_motivo
)

print("\n--- Predicción para Nueva Solicitud ---")
print(f"Estado de la Solicitud Predicho: {estado_predicho}")
if estado_predicho == 'rechazada':
    print(f"Motivo de Rechazo Predicho: {motivo_predicho}")


--- Predicción para Nueva Solicitud ---
Estado de la Solicitud Predicho: rechazada
Motivo de Rechazo Predicho: Prioridad baja en comparación con otras solicitudes
